# Wildcard Handling with Astroquery.mast

----

## Learning Goals

By the end of this tutorial, you will:

## Introduction

This notebook will demonstrate the use of wildcards in `astroquery.mast.Observations` criteria queries. The use of wildcards is encouraged for certain criteria types (`string` object types) to ensure that the results returned are not missing any observations. We will use 3 examples to demonstrate some use-cases for wildcards when doing criteria queries, and to emphasize on certain criteria where wildcard usage is highly encouraged, particularly for JWST queries. We will also use the last example to demonstrate the use of value ranges when working with `float` object criteria types.

The workflow for this notebook consists of:

* Wildcards with `astroquery.mast.Observations`
    * Wildcard Search with `instrument_name`
    * Wildcard Search with `instrument_name` and `proposal_id`
    * Wildcard Search a Time-sensitive Object with `target_name` and `t_min`
    * Resources

## Imports

In [83]:
from astropy.time import Time
from astroquery.mast import Observations

----

## Wildcards with `astroquery.mast.Observations`

The use of wildcards when making `astroquery.mast.Observations` queries can help ensure you retrieve all observations without leaving anything out. The available wildcards are `%` and `*` with `%` serving to replace a single character, and `*` serving to replace more than one character preceding, following, or in between the existing characters, depending on its placement. See the [Observation Criteria Queries](https://astroquery.readthedocs.io/en/latest/mast/mast.html#observation-criteria-queries) section in the `astroquery.mast` documentation for more information on the wildcards.

With that being said, wildcards are only available for certain criteria. Those criteria that are `string` type objects are criteria that accept wildcards, but `float`, `integer`, or any other objects do not accept wildcards.

Users may call the `get_metadata` method to see the list of criteria that can be queried with. The criteria listed as `string` objects under the **Data Type** column are criteria that can be called with wildcards:

In [46]:
Observations.get_metadata("observations").show_in_notebook()

idx,Column Name,Column Label,Data Type,Units,Description,Examples/Valid Values
0,intentType,Observation Type,string,,Whether observation is for science or calibration.,"Valid values: science, calibration"
1,obs_collection,Mission,string,,Collection,"E.g. SWIFT, PS1, HST, IUE"
2,provenance_name,Provenance Name,string,,"Provenance name, or source of data","E.g. TASOC, CALSTIS, PS1"
3,instrument_name,Instrument,string,,Instrument Name,"E.g. WFPC2/WFC, UVOT, STIS/CCD"
4,project,Project,string,,Processing project,"E.g. HST, HLA, EUVE, hlsp_legus"
5,filters,Filters,string,,Instrument filters,"F469N, NUV, FUV, LOW DISP, MIRROR"
6,wavelength_region,Waveband,string,,Energy Band,"EUV, XRAY, OPTICAL"
7,target_name,Target Name,string,,Target Name,Ex. COMET-67P-CHURYUMOV-GER-UPDATE
8,target_classification,Target Classification,string,,Type of target,Ex. COMET;COMET BEING ORBITED BY THE ROSETTA SPACECRAFT;SOLAR SYSTEM
9,obs_id,Observation ID,string,,"Observation identifier, given by mission","U24Z0101T, N4QF18030"


### Wildcard Search with `instrument_name`

For our first example we will search for all NIRISS observations taken by a certain proposal/program PI. For this case, our two criteria we query with are `proposal_pi` and `instrument_name`, which are both `string` object criteria. As such, both can be wildcarded for ease of use. In fact, there may be cases where it is necessary to wildcard the `instrument_name` variable. This is because the JWST science instrument names on MAST have been modified to include configuration information for more precise advanced searches. However, this may lead to an incomplete list of observations returned for those who are not trying to do an advanced search. 

We will demonstrate this by looking at the results for the query below:

In [157]:
observations = Observations.query_criteria(proposal_pi="Espinoza, Nestor",
                                           instrument_name="*NIRISS*")
observations

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str4,str7,str12,str4,str13,str8,str13,str1,str50,float64,float64,str8,str16,int64,float64,float64,float64,float64,float64,str70,float64,str4,str3,int64,str158,str63,str81,str6,bool,float64,str9,str9
science,JWST,CALJWST,NIRISS,JWST,F277W/GR700XD,Infrared,HAT-P-14,--,jw01541-o001_t002_niriss_f277w-gr700xd-substrip256,260.1161716666666,38.24216944444444,spectrum,"Espinoza, Nestor",3,59738.26121625,59738.55835982639,384.785,600.0,3200.0,NIRISS Sensitivity and Stability for Transiting Exoplanet Observations,59774.8541666,1541,COM,--,POLYGON 260.16189536 38.23724946 260.11337073 38.23746108 260.11348004 38.24619441 260.16201403 38.24597997 260.16189536 38.23724946,--,mast:JWST/product/jw01541-o001_t002_niriss_f277w-gr700xd-substrip256_x1dints.fits,PUBLIC,False,nan,85715273,195816449
science,JWST,CALJWST,NIRISS/SOSS,JWST,F480M,INFRARED,BD+60-1753,--,jw01512002001_02101_00001_nis,261.2178643508574,60.43078346928282,image,"Espinoza, Nestor",2,59995.98101357639,59995.98102200231,0.683,4600.0,5000.0,SOSS Wavelength and Trace,59996.18175925,1512,CAL,--,POLYGON 261.218693113 60.429993446 261.216484201 60.430416356 261.217327028 60.431499222 261.219536203 60.431076276,mast:JWST/product/jw01512002001_02101_00001_nis_cal.jpg,mast:JWST/product/jw01512002001_02101_00001_nis_cal.fits,PUBLIC,False,nan,117933629,255695241
science,JWST,CALJWST,NIRISS/SOSS,JWST,F480M,INFRARED,BD+60-1753,--,jw01512002001_02101_00004_nis,261.2178643508749,60.43078346928948,image,"Espinoza, Nestor",2,59995.98740097222,59995.98740939815,0.683,4600.0,5000.0,SOSS Wavelength and Trace,59996.18133101,1512,CAL,--,POLYGON 261.218665628 60.430005149 261.21645672 60.430428066 261.21729956 60.431510929 261.219508731 60.431087978,mast:JWST/product/jw01512002001_02101_00004_nis_cal.jpg,mast:JWST/product/jw01512002001_02101_00004_nis_cal.fits,PUBLIC,False,nan,117933631,255695252
science,JWST,CALJWST,NIRISS/SOSS,JWST,F480M,INFRARED,BD+60-1753,--,jw01512002001_02101_00003_nis,261.2178643508621,60.43078346928461,image,"Espinoza, Nestor",2,59995.98273135417,59995.98273978009,0.683,4600.0,5000.0,SOSS Wavelength and Trace,59996.18243051,1512,CAL,--,POLYGON 261.218532548 60.430043518 261.216323637 60.430466434 261.217166476 60.431549298 261.219375651 60.431126347,mast:JWST/product/jw01512002001_02101_00003_nis_cal.jpg,mast:JWST/product/jw01512002001_02101_00003_nis_cal.fits,PUBLIC,False,nan,117933632,255695267
science,JWST,CALJWST,NIRISS/IMAGE,JWST,CLEAR;GR700XD,INFRARED,BD+60-1753,--,jw01512002001_03101_00001_nis,261.2178643508796,60.43078346929124,image,"Espinoza, Nestor",2,59995.98902444445,59995.98915162037,5.494,500.0,5000.0,SOSS Wavelength and Trace,59996.02287034,1512,CAL,--,POLYGON 261.252875018 60.462615326 261.225284884 60.427069921 261.208823688 60.430220008 261.236381292 60.465773165,mast:JWST/product/jw01512002001_03101_00001_nis_trapsfilled.jpg,mast:JWST/product/jw01512002001_03101_00001_nis_rateints.fits,PUBLIC,False,nan,117933633,255695284
science,JWST,CALJWST,NIRISS/SOSS,JWST,F480M,INFRARED,BD+60-1753,--,jw01512002001_02101_00002_nis,261.2178643508597,60.4307834692837,image,"Espinoza, Nestor",2,59995.98186616898,59995.98187459491,0.683,4600.0,5000.0,SOSS Wavelength and Trace,59996.18305556,1512,CAL,--,POLYGON 261.218570044 60.429939833 261.216361139 60.430362749 261.217203974 60.431445613 261.219413143 60.431022662,mast:JWST/product/jw01512002001_02101_00002_nis_cal.jpg,mast:JWST/product/jw01512002001_02101_00002_nis_cal.fits,PUBLIC,False,nan,117933634,255695293
science,JWST,CALJWST,NIRISS/IMAGE,JWST,CLEAR;GR700XD,INFRARED,BD+60-1753,--,jw01512002001_03102_00001_nis,261.2178643508855,60.43078346929347,image,"Espinoza, Nesto

Here we have a list of all kinda of NIRISS observations led by the PI Dr. Espinoza. Let's call for all the unique values under the `instrument_name` column to see what was returned thanks to our `*` wildcards.

In [158]:
set(observations['instrument_name'])

{'NIRISS', 'NIRISS/IMAGE', 'NIRISS/SOSS'}

As you can see, there were some observations with `instrument_name` labeled something other than `NIRISS`, due to the new configuration on MAST. Had we only searched for the `instrument_name` value `NIRISS`, we would have missed several observations. For the full list of instrument name variations, you can refer to [this page](https://outerspace.stsci.edu/display/MASTDOCS/JWST+Instrument+Names).

### Wildcard Search with `instrument_name` and `proposal_id`

Let's repeat the previous example, but add another `string` criteria and wildcard into the mix. Let's query for all observations belonging to proposal/program IDs that begin with the digits `15`, and see what gets returned:

In [162]:
observations = Observations.query_criteria(proposal_pi='Espinoza, Nestor',
                                           instrument_name='*NIRISS*',
                                           proposal_id=['15%%'])
set(observations['proposal_id']), set(observations['instrument_name'])

({'1512', '1541'}, {'NIRISS', 'NIRISS/IMAGE', 'NIRISS/SOSS'})

**Note**: Be aware that you can be too generous with the wildcards. Too much ambiguity will lead to getting results that were not intended. See below for example.

In [80]:
observations = Observations.query_criteria(proposal_pi='Espinoza, Nestor',
                                           instrument_name='*NI*S*')
set(observations['instrument_name'])

{'NIRISS', 'NIRISS/IMAGE', 'NIRISS/SOSS', 'NIRSPEC/SLIT'}

This query returns `NIRSPEC/SLIT` observations in addition to the NIRISS ones, which is not what was intended.

### Wildcard Search a Time-sensitive Object with `target_name` and `t_min`

The criteria we will use for we will be querying for image observations of Comet 67P Churyumov-Gerasimenko observed through the Hubble Space Telescope's Advanced Camera for Surveys (ACS) Wide Field Camera (WFC). Since this is a moving target, we will not be able to query by coordinates, so we will work with the `target_name` criteria. This comet's name can be listed in different ways, so we will use `*` wildcards in our criteria query.

In [176]:
observations = Observations.query_criteria(target_name="*COM*67P*",
                                           instrument_name="ACS/WFC")

print(f"{len(observations)} total observations" + "\n")
print("Listed target names:")
print(set(observations['target_name']))

140 total observations

Listed target names:
{'COMET-67P-CHURYUMOV-GERASIMENK', 'COMET-67P-CHURYUMOV-GER-UPDATE'}


TODO: Add warning in doc to say that you may not actually be getting all your obs using target_name
because of all the variations of the target name

Alternatively, you can search for a moving target using RA and Dec, if you know the trajectory of the comet. Let's try to search for Comet 67P Churyumov Gerasimenko using the `t_min` column. 

But first let's see what we're working with in terms of the range in `t_min`. Remember, `t_min` is in MJD.

In [178]:
min_start, max_start = min(observations['t_min']), max(observations['t_min'])

min_start, max_start

(56887.26573422454, 57454.78809)

These are the minimum and maximum values of `t_min`, which means our range cannot go beyond this window of time. Let's take a look at these values in `iso` format.

In [179]:
Time(min_start, format='mjd').iso, Time(max_start, format='mjd').iso

('2014-08-18 06:22:39.437', '2016-03-07 18:54:50.976')

It looks like this comet was observed using ACS's Wide Field Camera from 2014 until 2016. Let's request for a smaller window of time, and work with just the year 2014 observations.

In [180]:
min_start_mjd, max_start_mjd = (Time('2014-08-18', format='iso').mjd, Time('2014-12-31', format='iso').mjd)
min_start_mjd, max_start_mjd

(56887.0, 57022.0)

Now that we've converted the times to MJD, let's feed it back into the query for the `t_min` criteria. Remember that you can feed intervals to the `t_min` criteria in list form, where the first element in the list is the lower limit, and the second element is the upper limit.

In [182]:
observations = Observations.query_criteria(target_name='*COM*67P*',
                                           dataproduct_type='image',
                                           t_min=[min_start_mjd, max_start_mjd])
observations

intentType,obs_collection,provenance_name,instrument_name,project,filters,wavelength_region,target_name,target_classification,obs_id,s_ra,s_dec,dataproduct_type,proposal_pi,calib_level,t_min,t_max,t_exptime,em_min,em_max,obs_title,t_obs_release,proposal_id,proposal_type,sequence_number,s_region,jpegURL,dataURL,dataRights,mtFlag,srcDen,obsid,objID
str7,str3,str6,str7,str3,str12,str7,str30,str89,str36,float64,float64,str5,str14,int64,float64,float64,float64,float64,float64,str93,float64,str5,str3,int64,str1191,str86,str35,str6,bool,float64,str8,str9
science,HST,CALACS,ACS/WFC,HST,F775W,Optical,COMET-67P-CHURYUMOV-GERASIMENK,COMET; COMET-67P-CHURYUMOV-GERASIMENKO WHICH IS THE PRIMARY TARGET FOR ROSETTA SPACECRAFT,jcis06010,290.9112348543,-27.90621726673,image,"Hines, Dean C.",3,56979.505933368055,56979.51200949074,300.0,690.0,860.0,Imaging Polarimetry of the 67P/Churyumov-Gerasimenko with ACS: Supporting the Rosetta Mission,57344.94012718,13863,GO,--,POLYGON -69.07205442999998 -27.91834206 -69.072058317717648 -27.918294549551291 -69.070418239999981 -27.91807766 -69.072739870000021 -27.88970302 -69.103821919999973 -27.89381052 -69.103818044368452 -27.893858065783444 -69.105458039999974 -27.89407459 -69.103144859999986 -27.92244975 -69.07205442999998 -27.91834206 -69.07205442999998 -27.91834206,mast:HST/product/jcis06010_drz.jpg,mast:HST/product/jcis06010_drz.fits,PUBLIC,True,nan,24839235,220744505
science,HST,CALACS,ACS/WFC,HST,F775W,Optical,COMET-67P-CHURYUMOV-GERASIMENK,COMET; COMET-67P-CHURYUMOV-GERASIMENKO WHICH IS THE PRIMARY TARGET FOR ROSETTA SPACECRAFT,jcis04010,290.2735685756,-28.02183037337,image,"Hines, Dean C.",3,56977.44858376157,56977.454660219904,300.0,690.0,860.0,Imaging Polarimetry of the 67P/Churyumov-Gerasimenko with ACS: Supporting the Rosetta Mission,57342.8724883,13863,GO,--,POLYGON -69.710339019999992 -28.03460849 -69.710342387468657 -28.034498772208945 -69.708739630000025 -28.03435188 -69.70961256999999 -28.00591362 -69.740906 -28.00877878 -69.740902662524562 -28.008888572547743 -69.742505129999984 -28.0090351 -69.74164077 -28.03747355 -69.710339019999992 -28.03460849 -69.710339019999992 -28.03460849,mast:HST/product/jcis04010_drz.jpg,mast:HST/product/jcis04010_drz.fits,PUBLIC,True,nan,24839231,220744722
science,HST,CALACS,ACS/WFC,HST,F775W,Optical,COMET-67P-CHURYUMOV-GERASIMENK,COMET; COMET-67P-CHURYUMOV-GERASIMENKO WHICH IS THE PRIMARY TARGET FOR ROSETTA SPACECRAFT,jcis05010,290.5806735758,-27.96626079113,image,"Hines, Dean C.",3,56978.44466003472,56978.45073653935,300.0,690.0,860.0,Imaging Polarimetry of the 67P/Churyumov-Gerasimenko with ACS: Supporting the Rosetta Mission,57343.70451382,13863,GO,--,POLYGON -69.403442700000028 -27.97923304 -69.403444700846464 -27.979100547103432 -69.401826809999989 -27.97897202 -69.40225662 -27.95052573 -69.433575589999975 -27.95301089 -69.433573626058035 -27.953143446156233 -69.435191219999979 -27.95327161 -69.43476999 -27.98171799 -69.403442700000028 -27.97923304 -69.403442700000028 -27.97923304,mast:HST/product/jcis05010_drz.jpg,mast:HST/product/jcis05010_drz.fits,PUBLIC,True,nan,24839233,220745079
science,HST,CALACS,ACS/WFC,HST,F606W,Optical,COMET-67P-CHURYUMOV-GERASIMENK,COMET; COMET-67P-CHURYUMOV-GERASIMENKO WHICH IS THE PRIMARY TARGET FOR ROSETTA SPACECRAFT,jcis06020,290.9147892768,-27.90546497065,image,"Hines, Dean C.",3,56979.51550486111,56979.66734533565,6280.0,470.0,720.0,Imaging Polarimetry of the 67P/Churyumov-Gerasimenko with ACS: Supporting the Rosetta Mission,57344.94447902,13863,GO,--,POLYGON -69.051552329999993 -27.91547868 -69.051566623053915 -27.915304522529482 -69.048023170000022 -27.91483611 -69.048086300262 -27.914066875478145 -69.045198070000026 -27.91368501 -69.045253211462011 -27.913013321787403 -69.031596869999987 -27.91120688 -69.031681552501226 -27.910176912272057 -69.027714189999983 -27.90965175 -69.02772849552359 -27.909477898544647 -69.024182240000016 -27.90900847 -69.02653583 -27.88039823 -69.057861839999987 -27.88454222 -69.057847539106

In [188]:
import astropy.units as u
o = Observations.query_criteria(instrument_name="ACS/WFC", coordinates='280.9234007501509 -31.01619293249', radius=1.0*u.deg)

In [189]:
set(o['target_name'])

{'BIAS', 'COMET-67P-CHURYUMOV-GERASIMENK', 'DARK'}

In [184]:
set(observations['s_ra']), set(observations['s_dec'])

({280.9234007501509,
  280.9256508003316,
  280.9325907501443,
  280.9348408003252,
  280.94283075005023,
  280.94509075023745,
  280.95182074981915,
  280.9542165962,
  280.9542207499916,
  281.0749808366519,
  281.0773208868312,
  281.0845408866453,
  281.0868808868347,
  281.0951608865579,
  281.0975108367455,
  281.1045408863173,
  281.1070302812,
  281.1070408364967,
  281.2558788729,
  290.2735685756,
  290.27357746214466,
  290.2751774620947,
  290.2770693515,
  290.2770720459829,
  290.28081209582683,
  290.2938020457327,
  290.2972520455628,
  290.3000220953313,
  290.3133420951515,
  290.317202094845,
  290.3206520946842,
  290.5806735758,
  290.58067756432786,
  290.58229756426226,
  290.5842080034,
  290.58421214616294,
  290.5879921960003,
  290.6007621459412,
  290.6042521457815,
  290.60707214553975,
  290.62052219537316,
  290.6244321950754,
  290.62792219490746,
  290.9112348543,
  290.9112370596945,
  290.9128770096229,
  290.9147892768,
  290.91479174383323,
  290.91

## Resources

## Citations

## About This Notebook

If you have comments or questions on this notebook, please contact us through the Archive Help Desk e-mail at archive@stsci.edu. <br>
<br>
Author(s): Jenny V. Medina <br>
Keyword(s): Tutorial, astroquery, wildcards, moving, target <br>
Last Updated: Jun 2023